In [34]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/travis.earheart/.config/gcloud/legacy_credentials/travis.earheart@gmail.com"

In [35]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/Users/travis.earheart/.config/gcloud/legacy_credentials/travis.earheart@gmail.com


In [ ]:
!pip install requests

In [1]:
import requests
import os
import sys
import pandas as pd
import json

# Added to get previous hour
from datetime import datetime, timedelta

### Fetch data from Polygon API

In [2]:
# Get previous hour (UTC)
last_hour_date_time = datetime.now() - timedelta(hours = 1)
print(last_hour_date_time.strftime('%Y-%m-%d %H:%M:%S'))

2022-06-09 15:47:00


In [3]:
published_utc = "2022-06-09T20:00:00Z"

In [4]:
api_key = os.getenv("POLYGON_API_KEY")
next_url = f"https://api.polygon.io/v2/reference/news?published_utc={published_utc}&limit=1000&apiKey={api_key}"
print(next_url)

https://api.polygon.io/v2/reference/news?published_utc=2022-06-09T20:00:00Z&limit=1000&apiKey=zUqayLy0QMKE4MfqABhB_jophZQFjrWZ


In [5]:
news = []
count = 0
resp = requests.get(next_url)

#while resp.json()["next_url"]:
while True:
    resp = requests.get(next_url)
    if resp.ok:
        news += resp.json()
        count += resp.json()["count"]
        print(f"Count: {count}")
        if "next_url" in resp.json().keys():
            next_url = resp.json()["next_url"] + f"&apiKey={api_key}"
        else:
            break
    else:
        print(f"Request failed with {resp.status_code}")
        sys.exit(1)



Count: 6


In [6]:
print(type(resp.json()))
#print(resp.json())
df = pd.DataFrame(resp.json()['results'])

<class 'dict'>


In [7]:
df.head()

,id,publisher,title,author,published_utc,article_url,tickers,amp_url,image_url,description,keywords
0,xEjXPj3qiAqUv0_U4LtToba6bUfwSq2HX4LF3WtYEvs,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...","Soluna Holdings, Inc. Announces Monthly Divid...","Soluna Holdings, Inc.",2022-06-09T20:00:00Z,https://www.globenewswire.com/news-release/202...,"[SLNHP, SLNH]",https://www.globenewswire.com/news-release/202...,https://ml.globenewswire.com/Resource/Download...,"ALBANY, NY, June 09, 2022 (GLOBE NEWSWIRE) -...",[Dividend Reports and Estimates]
1,T3U6xXzj7NBtVbtOxQsSsELcTztc2YNMBPTBp7-28Jg,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",FormFactor Adds Dilution Refrigeration (DR) Sy...,"FormFactor, Inc.",2022-06-09T20:00:00Z,https://www.globenewswire.com/news-release/202...,[FORM],https://www.globenewswire.com/news-release/202...,https://ml.globenewswire.com/Resource/Download...,Company Establishes Industry’s Most Comprehens...,[Product / Services Announcement]
2,Qq-1FSQGIx-pGMRISdrextQfSdUKEbmVaPKU3cRYtrc,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",APOGEE MINERALS PROVIDES UPDATE ON TRANSACTION...,Apogee Minerals Ltd.,2022-06-09T20:00:00Z,https://www.globenewswire.com/news-release/202...,[APMI],https://www.globenewswire.com/news-release/202...,https://ml.globenewswire.com/Resource/Download...,/NOT FOR DISTRIBUTION TO UNITED STATES NEWS SE...,[Company Announcement]
3,LaxLOCY1TMcLZdHwwBPD6Y4D4XchyEKD0UiLi-peWuc,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...","Paltalk, Inc. Announces Acquisition of Visicom...","Paltalk, Inc.",2022-06-09T20:00:00Z,https://www.globenewswire.com/news-release/202...,[PALT],https://www.globenewswire.com/news-release/202...,https://ml.globenewswire.com/Resource/Download...,"JERICHO, NY, June 09, 2022 (GLOBE NEWSWIRE) ...",[Press releases]
4,HvCmdL9FrVV4ipAxgsTiJUYDd0hRdq0o7qWSJFa_aRY,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",Inventiva announces three scientific presentat...,INVENTIVA,2022-06-09T20:00:00Z,https://www.globenewswire.com/news-release/202...,[IVA],https://www.globenewswire.com/news-release/202...,https://ml-eu.globenewswire.com/Resource/Downl...,"Daix (France), Long Island City (New York, Uni...","[Other News, Press releases, European Regulato..."


In [8]:
publisher = set()
for pub in df['publisher']:
    publisher.add(pub['name'])

print(publisher)

{'GlobeNewswire Inc.'}


In [27]:
!pip install google-cloud-storage

     |████████████████████████████████| 106 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 29.3 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 46.5 MB/s eta 0:00:01
     |████████████████████████████████| 962 kB 44.6 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 66.3 MB/s eta 0:00:01
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)


In [28]:
from google.cloud import storage

In [30]:
def upload_blob_from_memory(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The contents to upload to the file
    # contents = "these are my contents"

    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

    print(
        f"{destination_blob_name} with contents {contents} uploaded to {bucket_name}."
    )

In [29]:
upload_blob_from_memory(polygonio-news-sentiment-test,test.csv,test.csv)

NameError: name 'upload_blob_from_memory' is not defined

In [ ]:
bucket.blob('upload_test/test.csv').upload_from_string(df.to_csv(), 'text/csv')

In [37]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [38]:
implicit()

IsADirectoryError: [Errno 21] Is a directory: '/Users/travis.earheart/.config/gcloud/legacy_credentials/travis.earheart@gmail.com'

### Validate data

### Send data to ML model

In [ ]:
# Write data to Firestore database
# When writing data to Firestore - add new field for sentiment

### ML Model

In [ ]:
!pip install transformers

In [ ]:
!pip3 install torch torchvision torchaudio

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

/Users/travis.earheart/opt/miniconda3/envs/poly/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import transformers
transformers.__version__

'4.19.3'

In [11]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [12]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(['growth is strong and we have plenty of liquidity.', 
               'there is a shortage of capital, and we need extra financing.',
              'formulation patents might protect Vasotec to a limited extent.'])

In [13]:
results

[{'label': 'Positive', 'score': 1.0},
 {'label': 'Negative', 'score': 0.9952379465103149},
 {'label': 'Neutral', 'score': 0.9979718327522278}]

In [14]:
headlines = df['title'].tolist()

In [15]:
results = nlp(headlines)

In [16]:
results

[{'label': 'Neutral', 'score': 0.9999409914016724},
 {'label': 'Neutral', 'score': 0.6153307557106018},
 {'label': 'Neutral', 'score': 0.9999986886978149},
 {'label': 'Neutral', 'score': 0.9990658164024353},
 {'label': 'Neutral', 'score': 0.9999850988388062},
 {'label': 'Neutral', 'score': 0.9999991655349731}]

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [23]:
tokenizer2 = AutoTokenizer.from_pretrained("ProsusAI/finbert",num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [24]:
nlp2 = pipeline("text-classification", model=model, tokenizer=tokenizer)
results2 = nlp2(['growth is strong and we have plenty of liquidity.', 
               'there is a shortage of capital, and we need extra financing.',
              'formulation patents might protect Vasotec to a limited extent.'])

In [25]:
results2

[{'label': 'neutral', 'score': 0.7334725260734558},
 {'label': 'neutral', 'score': 0.7869898080825806},
 {'label': 'neutral', 'score': 0.7128837704658508}]

### Run data through model

### Deploy API